# 7. Ringkasan Proyek & Siklus TDSP

Bagian ini merangkum bagaimana proyek **Prediksi Harga Mobil Bekas di Arab Saudi (Syarah.com)** diimplementasikan dengan kerangka kerja **Team Data Science Process (TDSP)**.

TDSP menekankan pendekatan **iteratif** dan **agile** dengan lima langkah utama:

1. Business Understanding
2. Data Acquisition & Understanding
3. Modeling
4. Deployment
5. Customer / Stakeholder Acceptance

---
## 7.1 Ringkasan per Tahap TDSP

### 1. Business Understanding

- **Tujuan bisnis:**
  - Membantu penjual dan pembeli mobil bekas menentukan **harga wajar** berdasarkan data historis listing di Syarah.com.
  - Menyediakan rekomendasi harga yang bisa diintegrasikan ke platform sebagai fitur pendukung keputusan.

- **Pertanyaan kunci:**
  - Faktor apa saja yang paling memengaruhi harga mobil bekas?
  - Seberapa akurat model memprediksi harga dibandingkan dengan harga aktual di pasaran?
  - Apakah model cukup stabil untuk digunakan sebagai referensi harga di platform?

- **Metrik keberhasilan:**
  - MAE dan MAPE di bawah batas tertentu (contoh: MAE ≤ 10% median harga, MAPE ≤ 20%).

---
### 2. Data Acquisition & Understanding

- **Sumber data:**
  - Data listing mobil bekas dari **Syarah.com** yang berisi informasi seperti merk, tipe, tahun, region, jarak tempuh, harga, dan spesifikasi teknis.

- **Langkah utama:**
  - Load dataset `UsedCarsSA_Unclean_EN.csv`.
  - Menyusun **data dictionary** (penjelasan tiap kolom).
  - Melakukan **quick data health check**: bentuk data, tipe data, statistik deskriptif.
  - Membuat ringkasan **nilai unik per kolom** untuk memahami keragaman kategori dan potensi feature.

- **Outcome:**
  - Gambaran awal tentang kualitas dan kelengkapan data.
  - Insight awal mengenai fitur penting seperti `Year`, `Mileage`, `Engine_Size`, `Region`, dll.

---
### 3. Data Preparation & Feature Engineering

- **Pembersihan data:**
  - Menangani nilai hilang pada `Engine_Size` dengan modus per `(Type, Year)` dan modus global.
  - Mengisi nilai hilang pada `Gear_Type` dan kolom kategorikal lain dengan modus / 'Unknown'.
  - Menghapus kolom yang tidak informatif: `Link`, `Condition`, `Negotiable`.
  - Menghapus duplikat dan mengonversi kolom `Price` menjadi numerik.
  - Menghapus outlier harga ekstrem berdasarkan kuantil (1% dan 99%).

- **Feature engineering:**
  - Menambahkan fitur turunan **`Car_Age` = 2025 - Year** sebagai representasi usia kendaraan.

- **EDA singkat:**
  - Visualisasi distribusi numerik (histogram).
  - Hubungan Year, Engine_Size, Mileage terhadap Price (scatter + regresi).

- **Outcome:**
  - Dataset bersih disimpan sebagai `UsedCarsSA_Clean.csv`, siap untuk tahap modeling.

---
### 4. Modeling

- **Setup modeling:**
  - Target: `Price`.
  - Fitur: semua kolom selain `Price` (termasuk `Car_Age`).
  - Split data: 80% train, 20% test.

- **Preprocessing:**
  - Numerik: imputasi median + StandardScaler.
  - Kategorikal: imputasi modus + One-Hot Encoding.
  - Semua diorkestrasi melalui `ColumnTransformer` dan `Pipeline`.

- **Model yang diuji:**
  - Linear Regression (baseline).
  - RandomForestRegressor.
  - GradientBoostingRegressor.
  - XGBRegressor (XGBoost).

- **Metrik evaluasi:**
  - MAE, MAPE, RMSE, dan R².

- **Outcome:**
  - Model tree-based (misalnya Random Forest / XGBoost) biasanya menunjukkan performa lebih baik dibanding baseline linear regression.
  - Model terbaik dipilih berdasarkan kombinasi MAE rendah, MAPE rendah, dan R² yang memadai.

---
### 5. Deployment

- **Langkah utama:**
  - Melatih kembali model terbaik (contoh: Random Forest) dengan pipeline penuh.
  - Menyimpan model sebagai `used_car_price_model_rf.joblib` menggunakan `joblib.dump`.
  - Membuat fungsi `predict_price(single_record: dict)` yang:
    - Menerima input satu mobil dalam bentuk dictionary.
    - Mengubahnya ke DataFrame, menyelaraskan dengan kolom training.
    - Menghasilkan prediksi harga dalam SAR.

- **Gambaran integrasi:**
  - Model bisa dibungkus dalam API (FastAPI / Flask) untuk digunakan oleh platform.
  - Bisa juga dijalankan sebagai batch job untuk scoring berkala.
  - Output dapat ditampilkan sebagai **rekomendasi harga** dan **rentang harga wajar** di UI.

---
### 6. Customer / Stakeholder Acceptance

- **Validasi metrik:**
  - Menghitung kembali MAE, MAPE, RMSE, dan R² pada data uji untuk bahan diskusi.

- **Analisis error:**
  - Membuat `error_df` berisi harga aktual, prediksi, Absolute Error, dan APE.
  - Melihat kasus dengan error terbesar untuk memahami limitasi model.
  - Menghitung rata-rata APE per `Type` dan `Region` untuk melihat segmen mana yang paling stabil atau paling bermasalah.

- **Acceptance & hand-off:**
  - Menyusun rekomendasi penggunaan: model sebagai alat bantu, bukan satu-satunya sumber kebenaran.
  - Menyusun dokumentasi teknis dan operasional.
  - Menyepakati rencana iterasi berikutnya (penambahan fitur, pengayaan data, dll.) jika diperlukan.

---
## 7.2 Keterkaitan dengan Prinsip Agile & TDSP

Kerangka TDSP yang digunakan di proyek ini selaras dengan prinsip Agile:

- **Iteratif dan inkremental:**
  - Setiap tahap (dari Business Understanding hingga Acceptance) memungkinkan umpan balik untuk kembali ke tahap sebelumnya.
  - Misalnya, hasil analisis error di tahap 6 bisa memicu iterasi baru di tahap 2 atau 3.

- **Kolaboratif:**
  - Business stakeholder dilibatkan di awal (definisi tujuan) dan di akhir (acceptance), bukan hanya di awal.

- **Adaptif terhadap perubahan:**
  - Jika kebutuhan bisnis berubah (misalnya fokus ke segmen mobil tertentu), scope proyek bisa disesuaikan dan siklus TDSP diulang.

---
## 7.3 Limitasi Model & Data

Beberapa limitasi penting yang perlu diketahui:

1. **Keterbatasan fitur:**
   - Model hanya menggunakan fitur tabular dari listing (merk, tahun, mileage, dll.).
   - Tidak memperhitungkan kondisi fisik yang terlihat (baret, interior, modifikasi), karena tidak ada data gambar/teks deskriptif yang kaya.

2. **Distribusi data:**
   - Segmen mobil yang sangat jarang muncul dalam data bisa memiliki error besar.
   - Region dengan sedikit data juga rentan terhadap error tinggi.

3. **Dinamika pasar:**
   - Harga pasar mobil bekas bisa berubah karena faktor musiman, regulasi, atau ekonomi.
   - Model perlu diperbarui secara berkala agar tetap relevan.

---
## 7.4 Rekomendasi Next Step

Beberapa ide pengembangan lanjutan:

1. **Pengayaan Fitur**
   - Menambahkan fitur kondisi (misalnya skor inspeksi, informasi servis, atau deskripsi teks yang diproses dengan NLP).
   - Memasukkan informasi waktu (bulan/tahun listing) untuk menangkap pola musiman.

2. **Modeling Lanjutan**
   - Hyperparameter tuning yang lebih ekstensif (GridSearch / RandomizedSearch / Bayesian Optimization).
   - Coba model lain seperti LightGBM, CatBoost, atau stacking beberapa model.

3. **Monitoring Produksi**
   - Membuat dashboard untuk memantau performa model (MAE/MAPE) dari waktu ke waktu.
   - Mendeteksi **data drift**: apakah distribusi fitur di data baru berubah jauh dari data training.

4. **Feedback Loop dari User**
   - Menyimpan interaksi pengguna (misalnya, berapa sering rekomendasi harga diubah oleh admin / seller).
   - Menggunakan feedback ini untuk memperbaiki data dan model di iterasi berikutnya.

---
## 7.5 Kesimpulan

- Proyek ini menunjukkan bagaimana kerangka **TDSP** dapat digunakan untuk mengelola proyek data science secara terstruktur dari awal hingga akhir.
- Dengan mengikuti langkah-langkah:
  - **(1) Business Understanding** → memastikan model menjawab kebutuhan nyata.
  - **(2) Data Acquisition & Understanding** → memahami kualitas dan batasan data.
  - **(3) Modeling** → membangun dan membandingkan beberapa model.
  - **(4) Deployment** → membuat model siap pakai di lingkungan operasional.
  - **(5) Customer / Stakeholder Acceptance** → memvalidasi solusi dengan pemangku kepentingan.

- Hasil akhirnya adalah sebuah pipeline prediksi harga mobil bekas yang:
  - Dapat diintegrasikan ke platform seperti Syarah.com.
  - Dapat terus ditingkatkan melalui iterasi seiring bertambahnya data dan perubahan kebutuhan bisnis.

Dengan dokumentasi ini, tim lain (data scientist, engineer, maupun stakeholder bisnis) dapat memahami alur proyek dan melanjutkan pengembangan tanpa harus mulai dari nol.


In [ ]:
# Placeholder: di sini kamu bisa menambahkan kode apa pun untuk ringkasan otomatis,
# misalnya menampilkan struktur akhir df_clean jika diperlukan.

print('Section 7 — Ringkasan TDSP untuk proyek ini.')